# Часть 3. Векторизация текста

**Цель:** преобразовать предобработанные тексты в числовые признаки для последующего обучения моделей.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
DATA_DIR = Path("../data/processed")

In [3]:
train_path = DATA_DIR / "data_train.csv"
val_path   = DATA_DIR / "data_val.csv"
test_path  = DATA_DIR / "data_test.csv"

In [4]:
train = pd.read_csv(train_path)
val   = pd.read_csv(val_path)
test  = pd.read_csv(test_path)

In [5]:
for df in (train, val, test):
    df["processed_text"] = df["processed_text"].fillna("").astype(str)

In [6]:
train.shape, val.shape, test.shape

((3630, 14), (428, 14), (503, 14))

In [7]:
X_train_text = train["processed_text"].values
X_val_text   = val["processed_text"].values
X_test_text  = test["processed_text"].values

In [8]:
y_train = train["target"].values if "target" in train.columns else None
y_val   = val["target"].values if "target" in val.columns else None
y_test  = test["target"].values if "target" in test.columns else None

In [9]:
len(X_train_text), len(X_val_text), len(X_test_text)

(3630, 428, 503)

## BoW
BoW формирует вектор признаков как частоты слов, без учета порядка
Используем только обучающую выборку для обучения векторизатора, затем применяем к val/test

In [10]:
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

In [11]:
bow = CountVectorizer(
    min_df=2,          # можно подкрутить, чтобы убрать редкие слова
    max_df=0.95
)

In [12]:
X_train_bow = bow.fit_transform(X_train_text)
X_val_bow   = bow.transform(X_val_text)
X_test_bow  = bow.transform(X_test_text)

In [13]:
len(bow.vocabulary_)

7942

In [14]:
X_train_bow.shape, X_val_bow.shape, X_test_bow.shape

((3630, 7942), (428, 7942), (503, 7942))

Сохранение

In [15]:
VEC_DIR  = Path("../models/vectorizers")
FEAT_DIR = Path("../data/features")

In [16]:
joblib.dump(bow, VEC_DIR / "bow.joblib")

['..\\models\\vectorizers\\bow.joblib']

In [17]:
sparse.save_npz(FEAT_DIR / "X_train_bow.npz", X_train_bow)
sparse.save_npz(FEAT_DIR / "X_val_bow.npz",   X_val_bow)
sparse.save_npz(FEAT_DIR / "X_test_bow.npz",  X_test_bow)

## TF-IDF

TF-IDF снижает вклад слов, которые встречаются часто, и повышает вклад информативных слов

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tfidf = TfidfVectorizer(
    min_df=2,
    max_df=0.95,
    ngram_range=(1, 2)  # биграммы часто дают буст на текстах
)

In [20]:
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_val_tfidf   = tfidf.transform(X_val_text)
X_test_tfidf  = tfidf.transform(X_test_text)

In [21]:
len(tfidf.vocabulary_)

27571

In [22]:
X_train_tfidf.shape

(3630, 27571)

Сохранение

In [23]:
joblib.dump(tfidf, VEC_DIR / "tfidf.joblib")

sparse.save_npz(FEAT_DIR / "X_train_tfidf.npz", X_train_tfidf)
sparse.save_npz(FEAT_DIR / "X_val_tfidf.npz",   X_val_tfidf)
sparse.save_npz(FEAT_DIR / "X_test_tfidf.npz",  X_test_tfidf)

## Word2Vec

Word2Vec обучается на корпусе текстов и строит вектор для каждого слова.
Для получения вектора текста используется усреднение векторов его слов.

In [24]:
from gensim.models import Word2Vec

In [25]:
corpus_tokens = [t.split() for t in X_train_text]

w2v_model = Word2Vec(
    sentences=corpus_tokens,
    vector_size=300,
    window=5,
    min_count=2,
    workers=4,
    epochs=30,
    sg=1  # skip-gram
)

In [26]:
len(w2v_model.wv.key_to_index), w2v_model.vector_size

(8271, 300)

In [27]:
def w2v(texts, model):
    dim = model.vector_size
    out = np.zeros((len(texts), dim), dtype=np.float32)

    for i, txt in enumerate(texts):
        tokens = txt.split()
        vecs = [model.wv[w] for w in tokens if w in model.wv]
        if vecs:
            out[i] = np.mean(vecs, axis=0)
        else:
            out[i] = np.zeros(dim, dtype=np.float32)
    return out

In [28]:
X_train_w2v = w2v(X_train_text, w2v_model)
X_val_w2v   = w2v(X_val_text, w2v_model)
X_test_w2v  = w2v(X_test_text, w2v_model)

X_train_w2v.shape

(3630, 300)

In [29]:
print(w2v_model.wv['tesla'])

[-0.17977998 -0.05201431  0.7299367  -0.16983773  0.12864201 -0.18763827
  0.6392232   0.4573925   0.35686222 -0.13720316 -0.3028439  -0.22199935
 -0.6007448  -0.32712448 -0.49208674 -0.27370358  0.01200585 -0.2498504
 -0.06460121  0.07269163 -0.52926236  0.0209351  -0.04519065  0.3709578
  0.25904587 -0.5009567  -0.09983045  0.50056744  0.07983129  0.26324806
  0.45939967  0.05264655  0.04747412 -0.37085444 -0.00581874 -0.06841763
  0.4499923  -0.19635087  0.00323013  0.25756392 -0.650694    0.01219968
 -0.12714562 -0.572049   -0.09764056  0.30051318  0.2741736   0.26244625
 -0.0860332   0.31298673 -0.31466112 -0.50458306 -0.45976683 -0.20944677
 -0.32475707  0.30947408  0.16227537  0.2280456  -0.06464218 -0.04549627
  0.18980214 -0.33818457  0.5314199   0.16144541  0.11750107 -0.26347956
  0.26724496 -0.1492689  -0.14443454  0.06898117  0.10876758  0.00506366
 -0.18039288  0.08966258 -0.00503675  0.15507908  0.24569005  0.0331107
  0.16734979 -0.20054838 -0.24379341  0.28085408  0.37

In [30]:
# ближайшие слова
w2v_model.wv.most_similar('apple')

[('intelligence', 0.5937744379043579),
 ('homepod', 0.572170078754425),
 ('airtag', 0.5532761216163635),
 ('music', 0.4969641864299774),
 ('снести', 0.48946940898895264),
 ('шнур', 0.4827805161476135),
 ('tv', 0.4810587167739868),
 ('keyboard', 0.4790112376213074),
 ('генеалогия', 0.4738845229148865),
 ('передумать', 0.47298169136047363)]

In [31]:
# ближайшие слова
w2v_model.wv.most_similar('смартфон')

[('шифроваться', 0.45820122957229614),
 ('яблочный', 0.4561236798763275),
 ('улавливать', 0.4410543143749237),
 ('мач', 0.4393463134765625),
 ('canon', 0.42983031272888184),
 ('подтверждаться', 0.4213932454586029),
 ('galaxy', 0.4164232015609741),
 ('ночник', 0.4155904948711395),
 ('прошивка', 0.41365525126457214),
 ('magsafe', 0.4036543071269989)]

In [32]:
# косинусная близость
w2v_model.wv.similarity('apple', 'iphone')

np.float32(0.29898223)

In [33]:
EMB_DIR  = Path("../models/embeddings")

In [34]:
w2v_model.save(str(EMB_DIR / "word2vec.model"))

np.save(FEAT_DIR / "X_train_w2v.npy", X_train_w2v)
np.save(FEAT_DIR / "X_val_w2v.npy",   X_val_w2v)
np.save(FEAT_DIR / "X_test_w2v.npy",  X_test_w2v)

## BERT

Используем мультиязычную модель, чтобы корректно работать с русским текстом.
Извлекаем вектор текста:
- cls: вектор токена [CLS]
- mean: среднее по всем токенам последнего слоя

In [35]:
import torch
from transformers import BertTokenizer, BertModel

c:\Users\Arina\VS Code Projects\social-media-analytics\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [37]:
MODEL_NAME = "bert-base-multilingual-cased"

In [38]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
bert = BertModel.from_pretrained(MODEL_NAME).to(device)
bert.eval()

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 303.56it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: bert-base-multilingual-cased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [39]:
bert.config.hidden_size

768

In [40]:
def bert_embeddings(texts, batch_size=16, max_len=128, pooling="cls"):
    vecs = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = list(texts[i:i+batch_size])

            enc = tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=max_len,
                return_tensors="pt"
            ).to(device)

            last_hidden = bert(**enc).last_hidden_state  # [B, T, H]

            if pooling == "cls":
                emb = last_hidden[:, 0, :]  # [B, H]
            elif pooling == "mean":
                mask = enc["attention_mask"].unsqueeze(-1)  # [B, T, 1]
                emb = (last_hidden * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1)
            else:
                raise ValueError("pooling must be 'cls' or 'mean'")

            vecs.append(emb.cpu().numpy())

    return np.vstack(vecs)

In [41]:
X_train_bert = bert_embeddings(X_train_text, pooling="cls", batch_size=16, max_len=128)
X_val_bert   = bert_embeddings(X_val_text,   pooling="cls", batch_size=16, max_len=128)
X_test_bert  = bert_embeddings(X_test_text,  pooling="cls", batch_size=16, max_len=128)

In [42]:
print(X_train_bert.shape, X_val_bert.shape, X_test_bert.shape)

(3630, 768) (428, 768) (503, 768)


In [43]:
np.save(FEAT_DIR / "X_train_bert_cls.npy", X_train_bert)
np.save(FEAT_DIR / "X_val_bert_cls.npy",   X_val_bert)
np.save(FEAT_DIR / "X_test_bert_cls.npy",  X_test_bert)

## Оценка тональности по тональному словарю 

Дополнительно оцениваем тональность текста поста по словарю/
В качестве интерпретируемого базового признака использован русскоязычный тональный словарь RuSentiLex

In [44]:
LEX_PATH = Path("../data/external/rusentilex.txt")

In [45]:
def load_rusentilex_txt(path):
    if not path.exists():
        raise FileNotFoundError(f"Нет файла: {path.resolve()}")

    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("!"):
                continue
            parts = [p.strip() for p in line.split(",")]
            if len(parts) < 5:
                continue
            rows.append(parts[:5])

    df = pd.DataFrame(rows, columns=["word", "pos", "lemma", "sentiment", "source"])

    df["lemma"] = df["lemma"].astype(str).str.lower().str.strip()
    df["sentiment"] = df["sentiment"].astype(str).str.lower().str.strip()

    df = df[~df["lemma"].str.contains(r"\s+", regex=True)]
    pos_set = set(df.loc[df["sentiment"] == "positive", "lemma"])
    neg_set = set(df.loc[df["sentiment"] == "negative", "lemma"])

    return pos_set, neg_set, df

In [46]:
pos_set, neg_set, rusenti_df = load_rusentilex_txt(LEX_PATH)

print("pos:", len(pos_set), "neg:", len(neg_set))
rusenti_df.head()

pos: 2790 neg: 7867


,word,pos,lemma,sentiment,source
0,аборт,Noun,аборт,negative,fact
1,абортивный,Adj,абортивный,negative,fact
2,абракадабра,Noun,абракадабра,negative,opinion
3,абсурд,Noun,абсурд,negative,opinion
4,абсурдность,Noun,абсурдность,negative,opinion


In [47]:
def lexicon_score(text, pos_set, neg_set):
    toks = str(text).split() 
    if not toks:
        return 0.0
    pos_cnt = sum(t in pos_set for t in toks)
    neg_cnt = sum(t in neg_set for t in toks)
    return (pos_cnt - neg_cnt) / max(1, len(toks))

In [48]:
for df in (train, val, test):
    df["sent_score"] = df["processed_text"].apply(lambda s: lexicon_score(s, pos_set, neg_set))

train[["processed_text", "sent_score"]].head(10)

,processed_text,sent_score
0,hugging face выпустить бесплатный курс ия аген...,0.076923
1,кастомизировать ес киберстандарт samsung блоки...,0.000000
2,банк россия снизить ключевой ставка это четвёр...,-0.037037
3,топ хоррор версия учёный фильм вызывать самый ...,-0.058252
4,доллар дать сервис валюта продолжать падать до...,0.062500
5,пять страна ес запрет выдача виза россиянин со...,0.000000
6,айтишник сделать общаться девушка вместо ия ба...,0.000000
7,женщина нью йорк париж билет прятаться туалет ...,0.121212
8,реддитор видеокарта полноценный железнодорожны...,0.027027
9,осень питер первый минус год центр погода фобо...,-0.045455


In [49]:
train.to_csv(DATA_DIR / "data_train_with_sent.csv", index=False)
val.to_csv(DATA_DIR / "data_val_with_sent.csv", index=False)
test.to_csv(DATA_DIR / "data_test_with_sent.csv", index=False)